## Avaliando [Crash Course](https://mxnet.apache.org/versions/master/api/python/docs/tutorials/getting-started/crash-course/2-create-nn.html)


### Step2

In [30]:
import mxnet as mx
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()

mx.__version__

'2.0.0'

## Step 2 - Create neural network
Uma camada com 3 entradas e 5 saidas, com função de ativação "RELU"


In [31]:
layer=nn.Dense(5,in_units=3,activation='relu')
layer

Dense(3 -> 5, Activation(relu))

In [32]:
layer.initialize()

#### dez linhas com três parametros

In [33]:
x=np.random.uniform(-1,1,(10,3))
x

array([[ 0.14249027,  0.992553  ,  0.01552546],
       [ 0.4350289 ,  0.49777293,  0.50776994],
       [ 0.00312066,  0.43449652,  0.95477307],
       [-0.6382015 ,  0.41838038, -0.05247569],
       [ 0.5279087 , -0.42079234,  0.05847442],
       [ 0.62447   ,  0.5142175 ,  0.3640207 ],
       [-0.16083395, -0.7643049 , -0.8628838 ],
       [ 0.24051642, -0.768295  ,  0.3062178 ],
       [-0.7361935 , -0.67883015, -0.57000107],
       [-0.6338862 ,  0.85584867, -0.3990729 ]])

In [34]:
layer(x)

array([[0.03360424, 0.        , 0.02343912, 0.        , 0.        ],
       [0.02263017, 0.        , 0.01309925, 0.        , 0.        ],
       [0.06221364, 0.        , 0.        , 0.        , 0.        ],
       [0.04547961, 0.00674662, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.01896633, 0.04952385, 0.01996692],
       [0.0072762 , 0.        , 0.02859517, 0.        , 0.        ],
       [0.        , 0.08802361, 0.00980606, 0.05853732, 0.02507172],
       [0.        , 0.02953839, 0.        , 0.05750454, 0.01846377],
       [0.        , 0.10286247, 0.        , 0.02448843, 0.00708198],
       [0.04651404, 0.        , 0.        , 0.        , 0.        ]])

In [35]:
layer.params

{'weight': Parameter (shape=(5, 3), dtype=float32),
 'bias': Parameter (shape=(5,), dtype=float32)}

#### pesos e bias de entrada de cada elemento da camada interna de 5 neuronios.

In [36]:
layer.weight.data()

array([[-0.04882415,  0.04012945,  0.0470582 ],
       [-0.05229164, -0.0676925 , -0.0323052 ],
       [ 0.05310137,  0.01655371, -0.03592447],
       [ 0.04129067, -0.06801514, -0.01529054],
       [ 0.02127955, -0.02256562, -0.0130344 ]])

In [37]:
layer.bias.data()


array([0., 0., 0., 0., 0.])

Chain layers into a neural network using nn.Sequential

Sequential é a ligação de camadas onde a saída de uma camada é a entrada de outra

In [38]:
net = nn.Sequential()

net.add(nn.Dense(5,in_units=3,activation='relu'),nn.Dense(25,activation='relu'),nn.Dense(2))

net

Sequential(
  (0): Dense(3 -> 5, Activation(relu))
  (1): Dense(-1 -> 25, Activation(relu))
  (2): Dense(-1 -> 2, linear)
)

In [39]:
net[1]

Dense(-1 -> 25, Activation(relu))

#### Custom neural network architecture flexibly

In [40]:
class Net(nn.Block):
    def __init__(self):
        super().__init__()
        
    def forward(self,x):
        return x

In [41]:
class MLP(nn.Block):
    def __init__(self):
        super().__init__()
        self.dense1=nn.Dense(5,activation='relu')
        self.dense2=nn.Dense(25,activation='relu')
        self.dense3=nn.Dense(2)
        
    def forward(self,x):
        layer1=self.dense1(x)
        layer2=self.dense2(layer1)
        layer3=self.dense3(layer2)
        return layer3

In [42]:
net=MLP()
net

MLP(
  (dense1): Dense(-1 -> 5, Activation(relu))
  (dense2): Dense(-1 -> 25, Activation(relu))
  (dense3): Dense(-1 -> 2, linear)
)

In [43]:
net.dense1.params

{'weight': Parameter (shape=(5, -1), dtype=float32),
 'bias': Parameter (shape=(5,), dtype=float32)}

#### Creating custom layers using Parameters

In [44]:
from mxnet.gluon import Parameter

weight=Parameter("custom_parameter_weight",shape=(5,-1))
bias=Parameter("custom_parameter_bias",shape=(5,-1))

weight,bias

(Parameter (shape=(5, -1), dtype=<class 'numpy.float32'>),
 Parameter (shape=(5, -1), dtype=<class 'numpy.float32'>))

Custom layer com função linear sem função de ativação. 

w*x+b

In [45]:
class custom_layer(nn.Block):
    def __init__(self,out_units, in_units=0):
        super().__init__()
        self.weight=Parameter("weight",shape=(in_units,out_units),allow_deferred_init=True)
        self.bias=Parameter("bias",shape=(out_units,),allow_deferred_init=True)
        
    def forward(self,x):
        return np.dot(x,self.weight.data())+self.bias.data()

In [46]:
dense=custom_layer(3,in_units=5)
dense.initialize()
dense(np.random.uniform(size=(4,5)))

array([[-0.11508378,  0.05732282,  0.05646604],
       [-0.12577441,  0.01897157,  0.07220405],
       [-0.10327102,  0.06207245, -0.00171962],
       [-0.15234423,  0.05715137,  0.03755413]])

### [LeNet](http://yann.lecun.com/exdb/lenet/)

In [47]:
class LeNet(nn.Block):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2D(channels=6,kernel_size=3,activation='relu')
        self.pool1=nn.MaxPool2D(pool_size=2,strides=2)
        self.conv2=nn.Conv2D(channels=16,kernel_size=3,activation='relu')
        self.pool2=nn.MaxPool2D(pool_size=2,strides=2)
        self.dense1=nn.Dense(120,activation='relu')
        self.dense2=nn.Dense(84,activation='relu')
        self.dense3=nn.Dense(10)
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=self.pool2(x)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.dense3(x)
        return x

lenet=LeNet()        

In [48]:
class LeNet_custom(nn.Block):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2D(channels=6,kernel_size=3,activation='relu')
        self.pool1=nn.MaxPool2D(pool_size=2,strides=2)
        self.conv2=nn.Conv2D(channels=16,kernel_size=3,activation='relu')
        self.pool2=nn.MaxPool2D(pool_size=2,strides=2)
        self.dense1=nn.Dense(120,activation='relu')
        self.dense2=nn.Dense(84,activation='relu')
        self.dense3=custom_layer(10,84)
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=self.pool2(x)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.dense3(x)
        return x

lenet_custom=LeNet_custom()        

In [49]:
image_data=np.random.uniform(-1,1,(1,1,28,28))

lenet.initialize()
lenet_custom.initialize()

print("LeNet:")
print(lenet(image_data))

print("Custom LeNet:")
print(lenet_custom(image_data))


LeNet:
[[ 0.00158851  0.00225161 -0.00199851  0.00044887 -0.0034833   0.00018681
   0.00155292 -0.00247814  0.00156157  0.001675  ]]
Custom LeNet:
[[ 0.0187931  -0.05228931  0.06258925  0.06796222 -0.05891575  0.02589507
   0.01206913  0.0316846   0.02088234 -0.01235059]]


In [50]:
lenet.conv1.weight.data().shape, lenet.dense1.bias.data().shape,lenet_custom.conv1.weight.data().shape, lenet_custom.dense1.bias.data().shape


((6, 1, 3, 3), (120,), (6, 1, 3, 3), (120,))

#### Using predefined (pretrained) architectures

[Gluon CV model zoo](https://cv.gluon.ai/model_zoo/index.html)

[Gluon NLP model zoo](https://nlp.gluon.ai/model_zoo/index.html)

In [51]:
from mxnet.gluon import model_zoo

net=model_zoo.vision.resnet50_v2(pretrained=True)
net.hybridize()

dummy_input=np.ones(shape=(1,3,224,224))
output=net(dummy_input)
output.shape


(1, 1000)

### Deciding the paradigm for your network

In [52]:
net_hybrid_seq=nn.HybridSequential()

net_hybrid_seq.add(nn.Dense(5,in_units=3,activation='relu'),
    nn.Dense(25,activation='relu'),nn.Dense(2))

net_hybrid_seq

HybridSequential(
  (0): Dense(3 -> 5, Activation(relu))
  (1): Dense(-1 -> 25, Activation(relu))
  (2): Dense(-1 -> 2, linear)
)

In [53]:
net_hybrid_seq.hybridize()

### Creating custom layers using Parameters(HbridBlocks API)

In [54]:
class CustomLayer(nn.HybridBlock):
    def __init__(self,out_units, in_units=-1):
        super().__init__()
        self.weight=Parameter("weight",shape=(in_units, out_units),allow_deferred_init=True)
        self.bias=Parameter("bias",shape=(out_units,),allow_deferred_init=True)
        
    def forward(self, x):
        print(self.weight.shape,self.bias.shape)
        return np.dot(x,self.weight.data())+self.bias.data()

    def infer_shape(self,x):
        print(self.weight.shape,x.shape)
        self.weight.shape=(x.shape[-1],self.weight.shape[1])


dense=CustomLayer(3)

dense.initialize()
dense(np.random.uniform(size=(4,5)))
    

(-1, 3) (4, 5)
(5, 3) (3,)


array([[ 0.06865218,  0.04224551, -0.10759392],
       [ 0.04501069, -0.00186332, -0.0562923 ],
       [ 0.10729843,  0.06022658, -0.15761307],
       [ 0.06348709, -0.02355674, -0.07784321]])

### Performance

In [55]:
from time import time

def benchmark(net,x):
    y=net(x)
    start=time()
    for i in range(1,1000):
        y=net(x)
    return time()-start


x_bench=np.random.normal(size=(1,521))

net_hybrid_seq=nn.HybridSequential()

net_hybrid_seq.add (nn.Dense(256,activation='relu'),
                nn.Dense(128,activation='relu'),
                nn.Dense(2))

net_hybrid_seq.initialize()

print('Before hybridizing: %.4f sec'%(benchmark(net_hybrid_seq, x_bench)))
net_hybrid_seq.hybridize()
print('After hybridizing: %.4f'%(benchmark(net_hybrid_seq, x_bench)))

Before hybridizing: 0.5129 sec
After hybridizing: 0.1763


In [56]:
from mxnet.gluon import HybridBlock

class MLP_Hybrid(HybridBlock):
    def __init__(self):
        super().__init__()
        self.dense1=nn.Dense(256,activation='relu')
        self.dense2=nn.Dense(123,activation='relu')
        self.dense3=nn.Dense(2)

    def forward(self,x):
        layer1=self.dense1(x)
        layer2=self.dense2(layer1)
        layer3=self.dense3(layer2)
        return layer3

net_hybrid=MLP_Hybrid()
net_hybrid.initialize()

print('Before hybridizing: %.4f sec'%(benchmark(net_hybrid, x_bench)))
net_hybrid.hybridize()
print('After hybridizing: %.4f'%(benchmark(net_hybrid, x_bench)))

Before hybridizing: 0.4785 sec
After hybridizing: 0.1543


#### Saving and Loading your models

In [57]:
file_name='../../../data/crashCourse/models/layer.params'
layer.save_parameters(file_name)

In [58]:
def build_model():
    layer=nn.Dense(5,in_units=3,activation='relu')
    return layer

layer_new=build_model()

In [59]:
layer_new.load_parameters(file_name)

#### Save/load the model weights/parameters and the architectures

In [60]:
net_hybrid.export('../../../data/crashCourse/models/MLP_hybrid')

('../../../data/crashCourse/models/MLP_hybrid-symbol.json',
 '../../../data/crashCourse/models/MLP_hybrid-0000.params')

In [61]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    net_loaded=nn.SymbolBlock.imports('../../../data/crashCourse/models/MLP_hybrid-symbol.json',['data'], '../../../data/crashCourse/models/MLP_hybrid-0000.params',device=None)

In [62]:
net_loaded(x_bench)

array([[-0.08698318, -0.07087833]])

#### Visualizing Models

In [63]:
layer.summary(x)

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                                     (10, 3)               0
        Activation-1                                     (10, 5)               0
             Dense-2                                     (10, 5)              20
Parameters in forward computation graph, duplicate included
   Total params: 20
   Trainable params: 20
   Non-trainable params: 0
Shared params in forward computation graph: 0
Unique parameters in model: 20
--------------------------------------------------------------------------------


In [64]:
lenet.summary(image_data)

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                              (1, 1, 28, 28)               0
        Activation-1                              (1, 6, 26, 26)               0
            Conv2D-2                              (1, 6, 26, 26)              60
         MaxPool2D-3                              (1, 6, 13, 13)               0
        Activation-4                             (1, 16, 11, 11)               0
            Conv2D-5                             (1, 16, 11, 11)             880
         MaxPool2D-6                               (1, 16, 5, 5)               0
        Activation-7                                    (1, 120)               0
             Dense-8                                    (1, 120)           48120
        Activation-9                                     (1, 84)               0
            Dense-10        

In [65]:
net_hybrid_summary=MLP_Hybrid()

net_hybrid_summary.initialize()

net_hybrid_summary.summary(x_bench)

net_hybrid_summary.hybridize()

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                                    (1, 521)               0
        Activation-1                                    (1, 256)               0
             Dense-2                                    (1, 256)          133632
        Activation-3                                    (1, 123)               0
             Dense-4                                    (1, 123)           31611
             Dense-5                                      (1, 2)             248
        MLP_Hybrid-6                                      (1, 2)               0
Parameters in forward computation graph, duplicate included
   Total params: 165491
   Trainable params: 165491
   Non-trainable params: 0
Shared params in forward computation graph: 0
Unique parameters in model: 165491
---------------------------------------------------